# Classification Problem -- SVM -- Support Vector Machines

**Pros**: Low generalization error, computationally inexpensive, easy to interpret results.

**Cons**: Sensitive to tuning parameters and kernel choice; natively only handles binary classification.

**Works with**: Numeric values, nominal values.

## 1. Basic Knowledge
### 1.1 Separating data with the maximum margin

Let's assume that there are two groups of data, and the data points are separated enough that you could draw a straight line on the figure with all the points of one class on one side of the line and all the points of the other class on the other side of the line. If such a situation exists, we say the data is **linearly separable**(线性可分). And the line used to separate the dataset is called a **separating hyperplane**(分割超平面). 

If we have data with 1024 dimensions, we need something with 1023 dimensions to separate the data, which is called a **hyperplane**. The hyperplane is our decision boundary.

In order to make prediction results the most believable, we’d like to find the point closest to the separating hyperplane and make sure this is as far away from the separating line as possible(找最近的点使得距离分割面最远). This is known as **margin**. We want to have the greatest possible margin, because if we made a mistake or trained our classifier on limited data, we’d want it to be as robust as possible.

<br>
<font color='red'>The points closest to the separating hyperplane are known as **support vectors**. Now that we know that we’re trying to maximize the distance from the separating line to the support vectors, we need to find a way to optimize this problem.</font>

### 1.2 Finding the maximum margin

Our separating hyperplane has the form $w^Tx+b$. If we want to find the distance from A to the separating plane, we must measure normal or perpendicular(法线或垂线) to the line. This is given by $$\frac{|w^Tx+b|}{||w||}$$. 

We’re going to use something like the Heaviside step function(单位阶跃函数), $f(w^Tx+b)$, where the function $f(u)$ gives us -1 if u<0, and 1 otherwise. This is different from logistic regression in the previous chapter where the class labels were 0 or 1. 


The reason why we switched from class labels of 0 and 1 to -1 and 1 is that we can write a single equation to describe the margin or how close a data point is to our separating hyperplane and not have to worry if the data is in the -1 or +1 class.

The goal now is to find the w and b values that will define our classifier. To do this, we must find the points with the smallest margin. Then, when we find the points with the smallest margin, we must maximize that margin. This can be written as $$argmax_{w,b}\{min_n(label * (w^Tx+b))* \frac{1}{||w||}\}$$

其中$label*(w^Tx+b)$当点距离分割面距离很远的是时候，永远是一个很大的正数，无论是-1类还是1类。

Here, our constraint is that $label*(w^Tx+b)$ will be 1.0 or greater. There’s a well-known method for solving these types of constrained optimization problems, using something called **Lagrange multipliers**.

The optimization function turns out to be <font color='red'>$$max_{\alpha}[\sum_{i=1}^{m}{\alpha - \frac{1}{2}\sum_{i,j=1}^{m}{label^{(i)} *label^{(j)} * a_i * a_j * <x^{(i)}, x^{(j)}>}}]$$</font>

subject to the following constraints:
<font color='red'>$$\alpha \geq 0, and \sum_{i=1}^{m}{\alpha_i*label^{(i)}} = 0$$</font>

However, it needs one assumption: the data is 100% linearly separable. We know by now that our data is hardly ever that clean. With the introduction of something called **slack variables**(松弛变量), we can allow examples to be on the wrong side of the decision boundary. Our optimization goal stays the same, but we now have a new set of constraints:
$$C \geq \alpha \geq 0, and \sum_{i=1}^{m}{\alpha_i*label^{(i)}} = 0$$

The constant C controls weighting between our goal of making the margin large and ensuring that most of the examples have a functional margin of at least 1.0. The constant C is an argument to our optimization code that we can tune and get different results. 

## 2. Efficient optimization with the SMO algorithm

The SMO algorithm works to find a set of alphas and b. Once we have a set of alphas, we can easily compute our weights w and get the separating hyperplane.

Here’s how the SMO algorithm works: it chooses two alphas to optimize on each cycle. Once a suitable pair of alphas is found, one is increased and one is decreased. To be suitable, a set of alphas must meet certain criteria. One criterion a pair must meet is that both of the alphas have to be outside their margin boundary. The second criterion is that the alphas aren’t already clamped or bounded.

### 2.1 Solving small datasets with the simplified SMO

The outer loops of the Platt SMO algorithm determine the best alphas to optimize. We’ll skip that for this simplified version and select pairs of alphas by first going over every alpha in our dataset. Then, we’ll choose the second alpha randomly from the remaining alphas.

To do this we’re going to create a helper function that randomly selects one inte- ger from a range. We also need a helper function to clip values if they get too big. 

In [1]:
import numpy as np
from time import sleep

def load_dataset(fileName):
    data_mat = []
    label_mat = []
    fr = open(fileName)
    for line in fr.readlines():
        line_arr = line.strip().split('\t')
        data_mat.append([float(line_arr[0]), float(line_arr[1])])
        label_mat.append(float(line_arr[2]))
    return data_mat, label_mat

def select_jrand(i,m):
    j=i                                    # we want to select any j not equal to i
    while (j==i):
        j = int(np.random.uniform(0,m))       # m是所有alpha的数目
    return j

def clip_alpha(aj, H, L):
    if aj > H: 
        aj = H
    if L > aj:
        aj = L
    return aj


select_jrand function, takes two values. The first one, i, is the index of our first alpha, and m is the total number of alphas. A value is randomly chosen and returned as long as it’s not equal to the input i.

clip_alpha function, clips alpha values that are greater than H or less than L. These three helper functions don’t do much on their own, but they’ll be useful in our classifier.

In [2]:
data_arr, label_arr = load_dataset('/Users/jbian/Desktop/CU-life/summerlearning/mlinaction/dataset/testSet_chap6.txt')
label_arr[0:10]

[-1.0, -1.0, 1.0, -1.0, 1.0, 1.0, 1.0, -1.0, -1.0, -1.0]

In [3]:
# The simplified SMO algorithm

def smo_simple(data_mat, class_labels, C, toler, max_iter):
    data_matrix = np.mat(data_mat)
    label_mat = np.mat(class_labels).transpose()
    b = 0
    m,n = np.shape(data_matrix)
    alphas = np.mat(np.zeros((m,1)))
    iter = 0
    while (iter < max_iter):
        alpha_pairs_changed = 0
        for i in range(m):
            fxi = float(np.multiply(alphas,label_mat).T*(data_matrix*data_matrix[i,:].T)) + b
            ei = fxi - float(label_mat[i])                  #if checks if an example violates KKT conditions
            if ((label_mat[i]*ei < -toler) and (alphas[i] < C)) or ((label_mat[i]*ei > toler) and (alphas[i] > 0)):
                j = select_jrand(i,m)
                fxj = float(np.multiply(alphas,label_mat).T*(data_matrix*data_matrix[j,:].T)) + b
                ej = fxj - float(label_mat[j])
                alpha_i_old = alphas[i].copy()
                alpha_j_old = alphas[j].copy();
                if (label_mat[i] != label_mat[j]):
                    L = max(0, alphas[j] - alphas[i])
                    H = min(C, C + alphas[j] - alphas[i])
                else:
                    L = max(0, alphas[j] + alphas[i] - C)
                    H = min(C, alphas[j] + alphas[i])
                if L==H: 
                    print("L==H")
                    continue
                eta = 2.0 * data_matrix[i,:]*data_matrix[j,:].T - data_matrix[i,:]*\
                      data_matrix[i,:].T - data_matrix[j,:]*data_matrix[j,:].T
                if eta >= 0:
                    print("eta>=0")
                    continue
                alphas[j] -= label_mat[j]*(ei - ej)/eta
                alphas[j] = clip_alpha(alphas[j],H,L)
                if (abs(alphas[j] - alpha_j_old) < 0.00001): 
                    print("j not moving enough")
                    continue
                alphas[i] += label_mat[j]*label_mat[i]*(alpha_j_old - alphas[j])
                                                                        # update i by the same amount as j
                                                                        # the update is in the oppostie direction
                b1 = b - ei- label_mat[i]*(alphas[i]-alpha_i_old)*data_matrix[i,:]*data_matrix[i,:].T \
                     - label_mat[j]*(alphas[j]-alpha_j_old)*data_matrix[i,:]*data_matrix[j,:].T
                b2 = b - ej- label_mat[i]*(alphas[i]-alpha_i_old)*data_matrix[i,:]*data_matrix[j,:].T \
                     - label_mat[j]*(alphas[j]-alpha_j_old)*data_matrix[j,:]*data_matrix[j,:].T
                if (0 < alphas[i]) and (C > alphas[i]): 
                    b = b1
                elif (0 < alphas[j]) and (C > alphas[j]): 
                    b = b2
                else: 
                    b = (b1 + b2)/2.0
                alpha_pairs_changed += 1
                print("iter: %d i:%d, pairs changed %d" % (iter,i,alpha_pairs_changed))
        if (alpha_pairs_changed == 0): 
            iter += 1
        else: 
            iter = 0
        print("iteration number: %d" % iter)
    return b,alphas


伪代码：

创建一个alpha向量并将其初始化为0向量 

当迭代次数小于最大迭代次数时(外循环)

    对数据集中的每个数据向量(内循环):
    
    如果该数据向量可以被优化:
    
        随机选择另外一个数据向量
        
        同时优化这两个向量
        
        如果两个向量都不能被优化，退出内循环
        
    如果所有向量都没被优化，增加迭代数目，继续下一次循环

In [ ]:
b, alphas = smo_simple(data_arr, label_arr, 0.6, 0.001, 40)
# Here, I hide the output, because it's too long to display

In [5]:
b

matrix([[-3.74467465]])

In [6]:
alphas[alphas>0]

matrix([[0.1159755 , 0.24078828, 0.35676378]])

## 2.2 Speeding up optimization with the full Platt SMO

In [7]:
# Support functions for full Platt SMO

def kernel_trans(X, A, k_tup):         #calc the kernel or transform data to a higher dimensional space
    m,n = np.shape(X)
    K = np.mat(np.zeros((m,1)))
    if k_tup[0]=='lin': 
        K = X * A.T                    #linear kernel
    elif k_tup[0]=='rbf':
        for j in range(m):
            delta_row = X[j,:] - A
            K[j] = delta_row * delta_row.T
        K = np.exp(K/(-1*k_tup[1]**2))     #divide in NumPy is element-wise not matrix like Matlab
    else: 
        raise NameError('Houston We Have a Problem -- That Kernel is not recognized')
    return K

class opt_struct:
    def __init__(self, data_mat, class_labels, C, toler, k_tup):  # Initialize the structure with the parameters 
        self.X = data_mat
        self.label_mat = class_labels
        self.C = C
        self.tol = toler
        self.m = np.shape(data_mat)[0]
        self.alphas = np.mat(np.zeros((self.m,1)))
        self.b = 0
        self.e_cache = np.mat(np.zeros((self.m,2)))              # first column is valid flag
        self.K = np.mat(np.zeros((self.m,self.m)))
        for i in range(self.m):
            self.K[:,i] = kernel_trans(self.X, self.X[i,:], k_tup)
        
        
def calc_ek(oS, k):
    fxk = float(np.multiply(oS.alphas,oS.label_mat).T*oS.K[:,k] + oS.b)
    ek = fxk - float(oS.label_mat[k])
    return ek
        
def select_j(i, oS, ei):               #this is the second choice -heurstic, and calcs Ej
    max_k = -1
    max_delta_e = 0
    ej = 0
    oS.e_cache[i] = [1,ei]             #set valid 
                                       #choose the alpha that gives the maximum delta E
    valid_e_cache_list = np.nonzero(oS.e_cache[:,0].A)[0]
    if (len(valid_e_cache_list)) > 1:
        for k in valid_e_cache_list:   #loop through valid e_cache values and find the one that maximizes delta E
            if k == i: 
                continue               #don't calc for i, waste of time
            ek = calc_ek(oS, k)
            delta_e = abs(ei - ek)
            if (delta_e > max_delta_e):
                max_k = k 
                max_delta_e = delta_e
                ej = ek
        return max_k, ej
    else:                              #in this case (first time around) we don't have any valid e_cache values
        j = select_jrand(i, oS.m)
        ej = calc_ek(oS, j)
    return j, ej

def update_ek(oS, k):#after any alpha has changed update the new value in the cache
    ek = calc_ek(oS, k)
    oS.e_cache[k] = [1,ek]
        

In [8]:
# Full Platt SMO optimization routine

def inner_l(i, oS):
    ei = calc_ek(oS, i)
    if ((oS.label_mat[i]*ei < -oS.tol)and(oS.alphas[i]<oS.C))or((oS.label_mat[i]*ei>oS.tol)and(oS.alphas[i]>0)):
        j,ej = select_j(i, oS, ei)                      #this has been changed from select_jrand
        alpha_i_old = oS.alphas[i].copy()
        alpha_j_old = oS.alphas[j].copy()
        if (oS.label_mat[i] != oS.label_mat[j]):
            L = max(0, oS.alphas[j] - oS.alphas[i])
            H = min(oS.C, oS.C + oS.alphas[j] - oS.alphas[i])
        else:
            L = max(0, oS.alphas[j] + oS.alphas[i] - oS.C)
            H = min(oS.C, oS.alphas[j] + oS.alphas[i])
        if L==H:
            print("L==H")
            return 0
        eta = 2.0 * oS.K[i,j] - oS.K[i,i] - oS.K[j,j]    #changed for kernel
        if eta >= 0: 
            print("eta>=0")
            return 0
        oS.alphas[j] -= oS.label_mat[j]*(ei - ej)/eta
        oS.alphas[j] = clip_alpha(oS.alphas[j],H,L)
        update_ek(oS, j)                                 #added this for the e_cache
        if (abs(oS.alphas[j] - alpha_j_old) < 0.00001): 
            print("j not moving enough")
            return 0
        oS.alphas[i] += oS.label_mat[j]*oS.label_mat[i]*(alpha_j_old - oS.alphas[j])
                                                         #update i by the same amount as j
                                                         #the update is in the oppostie direction
        update_ek(oS, i)                                 #added this for the e_cache                    
        b1 = oS.b-ei-oS.label_mat[i]*(oS.alphas[i]-alpha_i_old)*oS.K[i,i]-oS.label_mat[j]*(oS.alphas[j]-alpha_j_old)*oS.K[i,j]
        b2 = oS.b-ej-oS.label_mat[i]*(oS.alphas[i]-alpha_i_old)*oS.K[i,j]-oS.label_mat[j]*(oS.alphas[j]-alpha_j_old)*oS.K[j,j]
        if (0 < oS.alphas[i]) and (oS.C > oS.alphas[i]): 
            oS.b = b1
        elif (0 < oS.alphas[j]) and (oS.C > oS.alphas[j]): 
            oS.b = b2
        else: 
            oS.b = (b1 + b2)/2.0
        return 1
    else: 
        return 0

In [9]:
# Full Platt SMO outer loop

def smo_p(data_mat, class_labels, C, toler, max_iter, k_tup=('lin', 0)):
    oS = opt_struct(np.mat(data_mat), np.mat(class_labels).transpose(), C, toler, k_tup)
    iter = 0
    entire_set = True
    alpha_pairs_changed = 0
    while (iter < max_iter) and ((alpha_pairs_changed > 0) or (entire_set)):
        alpha_pairs_changed = 0
        if entire_set:                            # go over all
            for i in range(oS.m):        
                alpha_pairs_changed += inner_l(i,oS)
                print("fullSet, iter: %d i:%d, pairs changed %d" % (iter,i,alpha_pairs_changed))
            iter += 1
        else:                                     # go over non-bound (railed) alphas
            non_bound_i = np.nonzero((oS.alphas.A > 0) * (oS.alphas.A < C))[0]
            for i in non_bound_i:
                alpha_pairs_changed += inner_l(i,oS)
                print("non-bound, iter: %d i:%d, pairs changed %d" % (iter,i,alpha_pairs_changed))
            iter += 1
        if entire_set: 
            entire_set = False                    # toggle entire set loop
        elif (alpha_pairs_changed == 0): 
            entire_set = True  
        print("iteration number: %d" % iter)
    return oS.b, oS.alphas

In [10]:
data_arr, label_arr = load_dataset('/Users/jbian/Desktop/CU-life/summerlearning/mlinaction/dataset/testSet_chap6.txt')
b, alphas = smo_p(data_arr, label_arr, 0.6, 0.001, 40)

fullSet, iter: 0 i:0, pairs changed 1
fullSet, iter: 0 i:1, pairs changed 1
fullSet, iter: 0 i:2, pairs changed 2
fullSet, iter: 0 i:3, pairs changed 2
fullSet, iter: 0 i:4, pairs changed 3
fullSet, iter: 0 i:5, pairs changed 4
fullSet, iter: 0 i:6, pairs changed 5
fullSet, iter: 0 i:7, pairs changed 5
fullSet, iter: 0 i:8, pairs changed 6
fullSet, iter: 0 i:9, pairs changed 6
fullSet, iter: 0 i:10, pairs changed 6
fullSet, iter: 0 i:11, pairs changed 6
fullSet, iter: 0 i:12, pairs changed 6
fullSet, iter: 0 i:13, pairs changed 6
fullSet, iter: 0 i:14, pairs changed 6
fullSet, iter: 0 i:15, pairs changed 6
fullSet, iter: 0 i:16, pairs changed 6
fullSet, iter: 0 i:17, pairs changed 7
fullSet, iter: 0 i:18, pairs changed 8
fullSet, iter: 0 i:19, pairs changed 8
j not moving enough
fullSet, iter: 0 i:20, pairs changed 8
j not moving enough
fullSet, iter: 0 i:21, pairs changed 8
fullSet, iter: 0 i:22, pairs changed 8
fullSet, iter: 0 i:23, pairs changed 9
fullSet, iter: 0 i:24, pairs chang

Then, we could use this to classify things: Firstly, we need to get the hyperplane from the alphas. This involves calculating $w$s. 

In [11]:
def calc_w(alphas, data_arr, class_labels):
    X = np.mat(data_arr)
    label_mat = np.mat(class_labels).transpose()
    m,n = np.shape(X)
    w = np.zeros((n,1))
    for i in range(m):
        w += np.multiply(alphas[i]*label_mat[i], X[i,:].T)
    return w

In [12]:
ws = calc_w(alphas, data_arr, label_arr)
print(ws)

[[ 0.64158661]
 [-0.27254318]]


In [13]:
data_mat = np.mat(data_arr)
print(data_mat[0]*np.mat(ws)+b)
print(label_arr[0])

[[-1.05316509]]
-1.0


Therefore, we could see that the first record belongs to -1 class. And it coresponds to the real label.

## 3. Using kernels for more complex data

We could use something called a **kernel** to transform our data into a form that’s easily understood by our classifier, that is to transform the data from one feature space to another. Usually, this mapping goes from a lower-dimensional feature space to a higher-dimensional space.

This mapping from one feature space to another is done by a kernel. You can think of the kernel as a wrapper(包装器) or interface(接口) for the data to translate it from a difficult formatting to an easier formatting. 

<br>
<font color='red'>One great thing about the SVM optimization is that all operations can be written in terms of inner products(内积). We can replace the inner products with our **kernel functions** without making simplifications. Replacing the inner product with a kernel is known as the **kernel trick** or kernel substation.</font>

The **radial bias function**(径向基核函数) is a kernel that’s often used with support vector machines. A radial bias function is a function that takes a vector and outputs a scalar based on the vector’s distance. This distance can be either from 0,0 or from another vector. We’ll use the **Gaussian version**, which can be written as $$k(x,y) = exp(\frac{-||x-y||^2}{2\sigma^2})$$

where $\sigma$ is a user-defined parameter that determines the “reach,” or how quickly this falls off to 0.

Its realization has been included in kenel_trans function above.

In [14]:
def test_rbf(k1=1.3):
    data_arr,label_arr = load_dataset('/Users/jbian/Desktop/CU-life/summerlearning/mlinaction/dataset/testSetRBF_chap6.txt')
    b,alphas = smo_p(data_arr, label_arr, 200, 0.0001, 10000, ('rbf', k1))         #C=200 important
    dat_mat = np.mat(data_arr)
    label_mat = np.mat(label_arr).transpose()
    sv_ind = np.nonzero(alphas.A>0)[0]
    svs = dat_mat[sv_ind]                                                          #get matrix of only support vectors
    label_sv = label_mat[sv_ind];
    print("there are %d Support Vectors" % np.shape(svs)[0])
    m,n = np.shape(dat_mat)
    error_count = 0
    for i in range(m):
        kernel_eval = kernel_trans(svs,dat_mat[i,:],('rbf', k1))
        predict = kernel_eval.T * np.multiply(label_sv, alphas[sv_ind]) + b
        if np.sign(predict) != np.sign(label_arr[i]): 
            error_count += 1
    print("the training error rate is: %f" % (float(error_count)/m))
    data_arr,label_arr = load_dataset('/Users/jbian/Desktop/CU-life/summerlearning/mlinaction/dataset/testSetRBF2_chap6.txt')
    error_count = 0
    dat_mat = np.mat(data_arr)
    label_mat = np.mat(label_arr).transpose()
    m,n = np.shape(dat_mat)
    for i in range(m):
        kernel_eval = kernel_trans(svs,dat_mat[i,:],('rbf', k1))
        predict = kernel_eval.T * np.multiply(label_sv, alphas[sv_ind]) + b
        if np.sign(predict) != np.sign(label_arr[i]): 
            error_count += 1    
    print("the test error rate is: %f" % (float(error_count)/m))   
    

In [15]:
test_rbf()

fullSet, iter: 0 i:0, pairs changed 1
fullSet, iter: 0 i:1, pairs changed 1
fullSet, iter: 0 i:2, pairs changed 2
fullSet, iter: 0 i:3, pairs changed 3
fullSet, iter: 0 i:4, pairs changed 3
fullSet, iter: 0 i:5, pairs changed 4
fullSet, iter: 0 i:6, pairs changed 5
fullSet, iter: 0 i:7, pairs changed 5
fullSet, iter: 0 i:8, pairs changed 6
fullSet, iter: 0 i:9, pairs changed 6
fullSet, iter: 0 i:10, pairs changed 7
fullSet, iter: 0 i:11, pairs changed 7
fullSet, iter: 0 i:12, pairs changed 7
fullSet, iter: 0 i:13, pairs changed 8
fullSet, iter: 0 i:14, pairs changed 9
fullSet, iter: 0 i:15, pairs changed 10
fullSet, iter: 0 i:16, pairs changed 11
fullSet, iter: 0 i:17, pairs changed 12
fullSet, iter: 0 i:18, pairs changed 12
fullSet, iter: 0 i:19, pairs changed 12
fullSet, iter: 0 i:20, pairs changed 12
L==H
fullSet, iter: 0 i:21, pairs changed 12
fullSet, iter: 0 i:22, pairs changed 12
fullSet, iter: 0 i:23, pairs changed 13
fullSet, iter: 0 i:24, pairs changed 13
fullSet, iter: 0 i:2

the training error rate is: 0.020000
the test error rate is: 0.040000
